# Mycobacterium tuberculosis

## Através do GenBank

In [1]:
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio import Entrez
from Bio import SeqIO

genes = ["rpoB", "rrs", "efpA"]

#Aceder ao NCBI
Entrez.email = 'pg45963@uminho.pt'
handle = Entrez.efetch(db = "nucleotide", id = "AL123456.3", rettype = "gb", retmode = "text")
seq_record = SeqIO.read(handle, "gb")

#Criar um ficheiro
SeqIO.write(seq_record, f"mycobacterium_tuberculosis.gb", "gb")
handle.close()

#Aceder aos dados
info = SeqIO.read("mycobacterium_tuberculosis.gb", "gb") 

f = open(f"mycobacterium_tuberculosis.gb")
info = SeqIO.read(f, "gb")

#print(info)
print(f"Gene Id: {info.id}")
print(f"Description: {info.description}")

taxonomia = ''.join(f"{m} | " for m in info.annotations["taxonomy"])
print(f"Taxonomy: {taxonomia[:-2]}")

print()
for i, ref in enumerate(info.annotations["references"]): print(f"\t-------- REFERENCE {i + 1} --------\n\n{ref}")

Gene Id: AL123456.3
Description: Mycobacterium tuberculosis H37Rv complete genome
Taxonomy: Bacteria | Actinobacteria | Corynebacteriales | Mycobacteriaceae | Mycobacterium | Mycobacterium tuberculosis complex 

	-------- REFERENCE 1 --------

authors: Cole,S.T., Brosch,R., Parkhill,J., Garnier,T., Churcher,C., Harris,D., Gordon,S.V., Eiglmeier,K., Gas,S., Barry,C.E. 3rd, Tekaia,F., Badcock,K., Basham,D., Brown,D., Chillingworth,T., Connor,R., Davies,R., Devlin,K., Feltwell,T., Gentles,S., Hamlin,N., Holroyd,S., Hornsby,T., Jagels,K., Krogh,A., McLean,J., Moule,S., Murphy,L., Oliver,K., Osborne,J., Quail,M.A., Rajandream,M.A., Rogers,J., Rutter,S., Seeger,K., Skelton,J., Squares,R., Squares,S., Sulston,J.E., Taylor,K., Whitehead,S. and Barrell,B.G.
title: Deciphering the biology of Mycobacterium tuberculosis from the complete genome sequence
journal: Nature 393 (6685), 537-544 (1998)
medline id: 
pubmed id: 9634230
comment: Erratum:[Nature 1998 Nov 12;396(6707):190] Erratum:[Nature 199

In [12]:
#print(info.features)

data = []

for value in info.features:
    try:
        gene_name = value.qualifiers["gene"][0]
    except KeyError:
        continue
    if value.type == "CDS" and gene_name in genes:
        #print(value)
        print(f"Gene Name: {gene_name}")
        print("Type:", value.type)
        locus_tag = value.qualifiers["locus_tag"][0]
        print(f"Gene Locus: {locus_tag}")
        print(f"Location: {value.location}")
        protein_id = value.qualifiers["protein_id"][0]
        pt_product = value.qualifiers["product"][0]
        description = ''.join(f"Protein {protein_id} has the function: {pt_product}")
        protein = ''.join(value.qualifiers["translation"][0])
        print(f"{description}\n-------- Sequence --------\n{protein}")
        data.append([gene_name, locus_tag, value.type, value.location, protein_id, pt_product[:46], f"{protein[:20]}..."])
        print("\n\t--------------------//--------------------\n")
        
    elif value.type == "rRNA" and gene_name in genes:
        #print(value)
        print(f"Gene Name: {gene_name}")
        print("Type:", value.type)
        print(f"Location: {value.location}")
        prot_obt = value.qualifiers["product"][0]
        print(f"Protein Obtained: {prot_obt}")
        data.append([gene_name, "----", value.type, value.location, "----", prot_obt[:46], "----"])
        print("\n\t--------------------//--------------------\n")

Gene Name: rpoB
Type: CDS
Gene Locus: Rv0667
Location: [759806:763325](+)
Protein CCP43410.1 has the function: DNA-directed RNA polymerase (beta chain) RpoB (transcriptase beta chain) (RNA polymerase beta subunit)
-------- Sequence --------
MADSRQSKTAASPSPSRPQSSSNNSVPGAPNRVSFAKLREPLEVPGLLDVQTDSFEWLIGSPRWRESAAERGDVNPVGGLEEVLYELSPIEDFSGSMSLSFSDPRFDDVKAPVDECKDKDMTYAAPLFVTAEFINNNTGEIKSQTVFMGDFPMMTEKGTFIINGTERVVVSQLVRSPGVYFDETIDKSTDKTLHSVKVIPSRGAWLEFDVDKRDTVGVRIDRKRRQPVTVLLKALGWTSEQIVERFGFSEIMRSTLEKDNTVGTDEALLDIYRKLRPGEPPTKESAQTLLENLFFKEKRYDLARVGRYKVNKKLGLHVGEPITSSTLTEEDVVATIEYLVRLHEGQTTMTVPGGVEVPVETDDIDHFGNRRLRTVGELIQNQIRVGMSRMERVVRERMTTQDVEAITPQTLINIRPVVAAIKEFFGTSQLSQFMDQNNPLSGLTHKRRLSALGPGGLSRERAGLEVRDVHPSHYGRMCPIETPEGPNIGLIGSLSVYARVNPFGFIETPYRKVVDGVVSDEIVYLTADEEDRHVVAQANSPIDADGRFVEPRVLVRRKAGEVEYVPSSEVDYMDVSPRQMVSVATAMIPFLEHDDANRALMGANMQRQAVPLVRSEAPLVGTGMELRAAIDAGDVVVAEESGVIEEVSADYITVMHDNGTRRTYRMRKFARSNHGTCANQCPIVDAGDRVEAGQVIADGPCTDDGEMALGKNLLVAIMPWEGHNYEDAIILSNRLVEEDVLTSIHIEEHEIDARDTKLG

## Matriz com as infos

In [14]:
import numpy as np
import pandas as pd
from tabulate import tabulate

info_firstline = ["Gene Name", "Gene Locus", "Type", "Location", "Protein ID", "Protein Function", "Protein Seq"]

numpy_data = np.array(data)
numpy_data = numpy_data.reshape((3, 7))

info_genes = pd.DataFrame(data = numpy_data, columns = info_firstline)
print(tabulate(info_genes, headers='keys', tablefmt='psql'))


+----+-------------+--------------+--------+----------------------+--------------+------------------------------------------------+-------------------------+
|    | Gene Name   | Gene Locus   | Type   | Location             | Protein ID   | Protein Function                               | Protein Seq             |
|----+-------------+--------------+--------+----------------------+--------------+------------------------------------------------+-------------------------|
|  0 | rpoB        | Rv0667       | CDS    | [759806:763325](+)   | CCP43410.1   | DNA-directed RNA polymerase (beta chain) RpoB  | MADSRQSKTAASPSPSRPQS... |
|  1 | rrs         | ----         | rRNA   | [1471845:1473382](+) | ----         | Ribosomal RNA 16S                              | ----                    |
|  2 | efpA        | Rv2846c      | CDS    | [3153038:3154631](-) | CCP45647.1   | Possible integral membrane efflux protein EfpA | MTALNDTERAVRNWTAGRPH... |
+----+-------------+--------------+--------+--------

## Obtenção do ficheiro xml com info do blast

### Place in the input which gene you want to blast and retrieve the information

In [9]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW 

while True:
    try:
        x = input("Which gene are you looking for (rpoB, rrs or efpA):")
        if x not in ["rpoB", "rrs", "efpA"]: 
            raise ValueError("Inserted gene not valid!")
        break
    except ValueError as error:
        print(error)

f = open(f"{x}.fasta")
seq = SeqIO.read(f, "fasta")

result_handle = NCBIWWW.qblast("blastn", "nt", seq.format("fasta"))
save_file = open(f"{x}_blast.xml", "w")
save_file.write(result_handle.read())
save_file.close()
result_handle.close()

Which gene are you looking for (rpoB, rrs or efpA): rrs


In [ ]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW 

while True:
    try:
        x = input("Which gene are you looking for (rpoB, rrs or efpA):")
        if x not in ["rpoB", "rrs", "efpA"]: 
            raise ValueError("Inserted gene not valid!")
        break
    except ValueError as error:
        print(error)

#f = open("/home/alexandreesperana/Documents/Apontamentos/Laboratorios_Bioinformatica/TPratico/rpoB.fasta")
f = open(f"{x}.fasta")
seq = SeqIO.read(f, "fasta")

result_handle = NCBIWWW.qblast("blastn", "nt", seq.format("fasta"))
save_file = open(f"{x}_blast.xml", "w")
save_file.write(result_handle.read())
save_file.close()
result_handle.close()

## Análise Ficheiro xml

In [10]:
from Bio.Blast import NCBIXML
import json

result_handle = open(f"{x}_blast.xml")
blast_records = NCBIXML.parse(result_handle)

allhits = []
#E_VALUE_THRESH = 0.05
for blast_record in blast_records:
    for alignment in blast_record.alignments: 
        for hsp in alignment.hsps:
            dict_all = {}
            dict_all["Info"] = alignment.hit_def
            dict_all["Hit"] = alignment.hit_id
            dict_all["Lenght"] = alignment.length
            dict_all["E-value"] = hsp.expect
            dict_all["Accession Number"] = alignment.accession
            allhits.append(dict_all)

print(f"Number of hits: {len(allhits)}")
print(json.dumps(allhits, indent = 3))

Number of hits: 50
[
   {
      "Info": "Mycobacterium tuberculosis strain FDAARGOS_756 chromosome, complete genome",
      "Hit": "gi|1848633072|gb|CP054014.1|",
      "Lenght": 4414577,
      "E-value": 0.0,
      "Accession Number": "CP054014"
   },
   {
      "Info": "Mycobacterium tuberculosis strain FDAARGOS_757 chromosome, complete genome",
      "Hit": "gi|1848629083|gb|CP054013.1|",
      "Lenght": 4417931,
      "E-value": 0.0,
      "Accession Number": "CP054013"
   },
   {
      "Info": "Mycobacterium tuberculosis strain TCDC3 chromosome",
      "Hit": "gi|1795949834|gb|CP047258.1|",
      "Lenght": 4413983,
      "E-value": 0.0,
      "Accession Number": "CP047258"
   },
   {
      "Info": "Mycobacterium tuberculosis strain TCDC7 chromosome",
      "Hit": "gi|1790790270|gb|CP047163.1|",
      "Lenght": 4641184,
      "E-value": 0.0,
      "Accession Number": "CP047163"
   },
   {
      "Info": "Mycobacterium tuberculosis strain TCDC10 chromosome",
      "Hit": "gi|17907902